<a href="https://colab.research.google.com/github/petalzx/amazonReviewScraper/blob/main/amazonReviewScaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Amazon Review Web Scaper

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [ ]:
#Modify the search_query in order to create data set from a different search
search_query = 'headphones'
base_url = 'https://www.amazon.com/s?k='

In [ ]:
url = base_url + search_query
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 
          'referer':'https://www.amazon.com/s?k=headphones&crid=3RIA528VORR9E&sprefix=headphones%2Caps%2C107&ref=nb_sb_noss_2'}

In [ ]:
#generating search response
response = requests.get(url, headers=header)
response.status_code

200

In [ ]:
#scape Amazon reviews of products that pop up from a specific search_query
def search(search_query):
  url = 'https://www.amazon.com/s?k=' + search_query
  print(url)
  response = requests.get(url, headers=header)
  if response.status_code==200:
    return response
  return "Error"

In [ ]:
#scape Amazon reviews of individual product pages using asin indentification number
def asinSearch(asin):
  url = 'https://www.amazon.com/dp/' + asin
  print(url)
  response = requests.get(url, headers=header)
  if response.status_code==200:
    return response
  return "Error"

In [ ]:
#brings us to "see all reviews" page
def reviewSearch(review_url):
  url = "https://www.amazon.com" + review_url
  print(url)
  response = requests.get(url, headers=header)
  if response.status_code==200:
    return response
  return "Error"

In [ ]:
product_names = []
response = search('headphones')
soup = bs(response.content)
for i in soup.findAll('span', {'class':'a-size-medium a-color-base a-text-normal'}):
  product_names.append(i.text)

https://www.amazon.com/s?k=headphones


In [ ]:
product_price = []
for i in soup.findAll('span', {'class':'a-offscreen'}):
  product_price.append(i.text)

In [ ]:
product_asin = []
for i in soup.findAll('div',{'class':'sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16'}):
  product_asin.append(i['data-asin'])

In [ ]:
product_review_url = []
for i in range(len(product_asin)):
  response = asinSearch(product_asin[i])
  soup = bs(response.content)
  product_review_url.append(soup.find('a',{'class':'a-link-emphasis a-text-bold'}).get('href'))

https://www.amazon.com/dp/B07YVYZ8T5
https://www.amazon.com/dp/B08528YFM3
https://www.amazon.com/dp/B00NJ2M33I
https://www.amazon.com/dp/B01M0GB8CC
https://www.amazon.com/dp/B094NC89P9
https://www.amazon.com/dp/B08VNFD8FS
https://www.amazon.com/dp/B0BLZ9PJVD
https://www.amazon.com/dp/B07NM3RSRQ
https://www.amazon.com/dp/B083S6Q8VK
https://www.amazon.com/dp/B095J62YQR
https://www.amazon.com/dp/B096SSCV5K
https://www.amazon.com/dp/B08WR6YDKP
https://www.amazon.com/dp/B09KGLRF8J
https://www.amazon.com/dp/B08G1XKY6N
https://www.amazon.com/dp/B09LYF2ST7
https://www.amazon.com/dp/B098FKXT8L


In [ ]:
#All of the data categories listed out
id_data = []
name_data = []
price_data = []
asin_data = []
#reviewer_name_data = []
title_data = []
text_data = []
score_data = []

In [ ]:
#collects data for all categories, loops through each product from the search and collects data from reviews of that product
def productScrape():
  #id_number = 0
  for i in range(len(product_review_url)):
    count = 0
    
    response = reviewSearch(product_review_url[i])
    soup = bs(response.content)

    score_raw = soup.find_all('i', class_=lambda x: x and 'review-rating' in x.split() and 'cr-lightbox-review-rating' not in x.split())
    title_raw = soup.find_all('a', {'data-hook':'review-title'})

    for j in soup.find_all('span',{'data-hook':'review-body'}):
      #id_data.append(id_number)
      #id_number+=1

      name_data.append(product_names[i])
      price_data.append(product_price[i])
      asin_data.append(product_asin[i])

      text_data.append(j.text)

      score_data.append(score_raw[count+2].text[:1])
      title_data.append(title_raw[count].text)
      count+=1

In [ ]:
productScrape()
data = {'Name':name_data, 'Price':price_data, 'Asin #':asin_data, #'Reviewer Name':reviewer_name_data,
        'Title':title_data, 'Text':text_data, 'Score':score_data}
df = pd.DataFrame(data)

https://www.amazon.com/Beats-Solo3-Wireless-Ear-Headphones/product-reviews/B07YVYZ8T5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Beats-Studio3-Wireless-Headphones-Collection/product-reviews/B08528YFM3/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Sony-MDRZX110-BLK-Stereo-Headphones/product-reviews/B00NJ2M33I/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Apple-EarPods-Lightning-Connector-White/product-reviews/B01M0GB8CC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/RORSOU-R10-Headphones-Microphone-Lightweight/product-reviews/B094NC89P9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Wireless-Bluetooth-Headphones-Foldable-Headsets/product-reviews/B08VNFD8FS/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
https://www.amazon.com/Bluetooth-Headphones-Waterproof-Earphones-Microphone/product-

In [ ]:
df.head(20)

,Name,Price,Asin #,Title,Text,Score
0,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nThis is like upgrading my entire brain. SO I...,\nHow did I not logically know that these woul...,5
1,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nFit of Headphones\n,\nThe headphones are great when it comes to so...,4
2,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nJust what I needed\n,\nMy AirPods kept falling out of my ears while...,5
3,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nPleasant surprise\n,\nI have had issues with wireless earbuds fitt...,5
4,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\neasy to use\n,\nThis is one item that I had to get to splurg...,4
5,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nGood for small ears!\n,\nI heard a lot of people complain because the...,5
6,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nLove this product\n,\nThis is a replacement pair. Had a pair a fe...,5
7,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nAwesome Product\n,\nGreat headphones for the price. You can adju...,5
8,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nBreaking them in matters\n,\nI have a love hate relationship with these. ...,5
9,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nDreBeats are EVERYTHING!!!!\n,\nAbsolutely LOVE these headphones!! Sound qua...,5


In [ ]:
df.sample(5)

,Name,Price,Asin #,Title,Text,Score
109,Wireless Earbuds Bluetooth Earbuds Wireless He...,$49.95,B096SSCV5K,\nA bit big for my ears\n,\nMy Apple EarPods had let me down in terms of...,4
2,Beats Solo3 Wireless On-Ear Headphones - Apple...,$131.39,B07YVYZ8T5,\nJust what I needed\n,\nMy AirPods kept falling out of my ears while...,5
92,JBL VIBE 100 TWS - True Wireless In-Ear Headph...,$29.95,B095J62YQR,\nMas o menos\n,\nTenia altisimas espectaciones ya que tengo a...,4
88,Srhythm NC25 Active Noise Cancelling Headphone...,$29.00,B083S6Q8VK,\nCalidad excelente por un buen precio\n,"\nMe ha sorprendido, la verdad. Es buena cance...",5
38,Apple EarPods Headphones with Lightning Connec...,$9.99,B01M0GB8CC,\nGreat product!\n,\nI’ve been trying to purchase these exact hea...,5


In [ ]:
#Saves the dataframe as a csv file
df.to_csv('amazon_data.csv', index=True)

In [ ]:
#Downloads csv file to local machine
from google.colab import files
files.download('amazon_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>